In [5]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import os
os.chdir("/Users/jundong/colloids-master/python")
from colloids import track, lif
from copy import deepcopy
from scipy import ndimage
from scipy.spatial.distance import cdist
from skimage.filters import threshold_otsu
%matplotlib inline

import contactAnalysis as ca
from contactAnalysis import *
import read_lif
import _pickle as cPickle
# import importlib
# importlib.reload(properties)

In [2]:
folders = ['/Users/jundong/PhD_3rd/Experiments/STED/2018_04_13/JD1803_1_11phic0_29Cp0_77_decon/',
           '/Users/jundong/PhD_3rd/Experiments/STED/2018_04_14/JD1803_1_11phic0_30Cp1_09/JD1803_1_11phic0_30Cp1_09_Series009_t0_decon/',
           '/Users/jundong/PhD_3rd/Experiments/STED/2018_04_14/JD1803_1_11phic0_29Cp1_40_pixel0_127/JD1803_1_11phic0_29Cp1_40_pixel0_127_Series019_decon/',       
           '/Users/jundong/PhD_3rd/Experiments/STED/2018_04_12/JD1803_1_11phic0_27Cp1_59_decon_t0/',
           '/Users/jundong/PhD_3rd/Experiments/STED/2018_04_05/JD1803-1_11_phic0_30Cp1_83_decon/',
           '/Users/jundong/PhD_3rd/Experiments/STED/2018_04_13/JD1803_1_11phic0_29Cp1_88_decon/',
           '/Users/jundong/PhD_3rd/Experiments/STED/2018_04_10/JD1803-1_11phic0_26Cp1_89_decon/',
           '/Users/jundong/PhD_3rd/Experiments/STED/2018_04_11/JD1803_1_11phic0_37Cp2_18_recheck/JD1803_1_11phic0_37Cp2_18_recheck_t0_decon/']

lif_names = ["JD1803_1_11phic0_29Cp0_77_decon.lif",
             "JD1803_1_11phic0_30Cp1_09_Series009_t0_decon.lif",
             "JD1803_1_11phic0_29Cp1_40_pixel0_127_Series019_decon.lif",
             "JD1803_1_11phic0_27Cp1_59_decon_t0.lif",
             "JD1803-1_11_phic0_30Cp1_83_decon.lif",
             "JD1803_1_11phic0_29Cp1_88_decon.lif",
             "JD1803-1_11phic0_26Cp1_89_decon.lif",
             "JD1803_1_11phic0_37Cp2_18_recheck_t0_decon.lif"]

In [3]:
folder_idx=[0,1,6]


In [4]:
#### Calculate g(r) to get the distance of the first minimum, to define neighbour distance
from tqdm import tqdm
from contactAnalysis import g_r
t=0
for f in folder_idx:
    print ('folder',f)
    folder = folders[f]
    lif_name = lif_names[f]
    
    gr_resutls = []
    for i in tqdm(range(10)):
        r,GR,IGR,IG=g_r.gr(folder+'t%01d/c1/particle_center.xyz'%t,1,1,1,screen=False)
        p_sigma, maxdistance = g_r.first_minimum(GR)
        gr_resutls.append((p_sigma,maxdistance))
    np.savetxt(folder+'t%01d/c1/gr_dparticle_dneighbour.txt'%t,max(gr_resutls))
    print ('particle diameter',max(gr_resutls)[0],'neighbour distance', max(gr_resutls)[1])

  0%|          | 0/10 [00:00<?, ?it/s]

folder 0


  0%|          | 0/10 [00:00<?, ?it/s]

particle diameter 21 neighbour distance 28
folder 1


  0%|          | 0/10 [00:00<?, ?it/s]

particle diameter 21 neighbour distance 26
folder 6


100%|██████████| 10/10 [00:27<00:00,  2.78s/it]

particle diameter 21 neighbour distance 26


In [ ]:
for f in folder_idx:
    print ('folder',f)
    t=0

    #### load both channel images (C1 and C2) from .lif file
    folder = folders[f]  
    reader = read_lif.Reader(folder+lif_names[f])
    series = reader.getSeries()
    image_xyz_c1 = series[0].getFrame(T=t)
    image_xyz_c2 = series[1].getFrame(T=t)
    
    imageC1=np.swapaxes(image_xyz_c1,0,2)      #### z, y, x
    imageC2=np.swapaxes(image_xyz_c2,0,2)      #### z, y, x
    print ('imageC2 shape is', imageC2.shape)

    #### Load coordinates of particles
    pcen_all = np.loadtxt(folder+"t%01d/c1/particle_center.txt"%t)
    print (pcen_all[:,1].max(), pcen_all[:,3].mean())
    pcen = deepcopy(pcen_all)
    
    # index of particles at edges and inside the box
    edge_particle, p_inbox = analysis.remove_edge_particles(imageC2,pcen_all)
    particle_inbox = pcen_all[p_inbox]

    p_cen = particle_inbox
    p_rad = particle_inbox[:,3]
    
    
    #### Calculate the distance of the first minimum in g(r)
    _gr = np.loadtxt(folder+'t%01d/c1/gr_dparticle_dneighbour.txt'%t)
    p_sigma, maxdistance = _gr[0],_gr[1]
    print ('particle diameter',p_sigma,'neighbour distance', maxdistance)

    
     #'###### Calculate volume fraction'
    from contactAnalysis import properties, figures
    sizeFactor = p_sigma/p_cen[:,3].mean()/2
    print (sizeFactor)
    phi,pd,z_first,z_last = properties.volume_fraction(p_cen,imageC2,factor=sizeFactor,z_range=True)
    print (phi,pd,z_first,z_last)

    figures.pd_hist(p_cen[:,3],folder+'t%01d/c1/'%t,bin_number=20)
    f = open(folder+'t%01d/c1/phic_pd_t%01d_removeEdges.txt'%(t,t),'w')
    f.write('%s %s %s' %('phi_c',phi,'\n')) 
    f.write('%s %s %s'%('polydispersity',pd,'\n'))
    f.close()
    
    
#     #### Do not consider the empty space in z direction
#     density_z = properties.phi_of_z(imageC2,folder,bin_z=30,T=1)
#     slices = np.array((int(density_z[1][0]),int(density_z[1][-1])))
#     print ('Validate Z slices between',slices)
    
    
    # apply Otsu filter twice to the contacts image
    M = imageC2
    slices=np.array((z_first, z_last))
#     slices = np.array((0,imageC2.shape[0]))
    imageC2_part = M[slices[0]:slices[1],:,:]
    print (imageC2_part.shape)
    background =  threshold_otsu(imageC2_part.ravel())
    threshold_contact=threshold_otsu(imageC2_part[imageC2_part>background].ravel())
    print ('background threshold', background)
    print ('contacts threshold', threshold_contact)
    
    fig.multifigures_xy3(M,120,200,400)        # Check the images with otsu filter
    # threshold images
    I2=deepcopy(M)
    threshold_C2 = ds.threshold_top_bottom(I2,180, threshold_contact*1.05)
    fig.multifigures_xy3(threshold_C2, 120, 200, 400)
    plt.savefig(folder+'threhsold.pdf')
    
    # find middle points between particles
    first_minimum = maxdistance
    neigbours_distance, n_p = ds.coordination(pcen,maxdistance=first_minimum)[:2]
    
    # neigbours_distance is a list of 
    #(particle, its neighbour, distances between these two particles)
    # n_p is the number of neighbours
    middle_mask,particle_middle = ds.middle_point(pcen,neigbours_distance,imageC2)[:2]
    
    
    mask_thr =0# threshold_contact
    spheres, dimmask,struct_size = ds.draw_spheres(middle_mask,imageC2, mask_thr,s_iter=3,dil_iter=1)
    labelled_spheres, num_spheres, sphere_size = ds.label_mask(spheres)
    print ('number of drawn spheres',num_spheres)
    print ('number of middle points',len(particle_middle)/2.)
    'The difference is connected spheres'
    plt.figure()
    plt.hist(sphere_size)
    plt.xlabel('Sphere size (pixels)'),plt.ylabel('Counts')
    plt.savefig(folder+'t%01d/c2/hist_sphereSizes.pdf'%t)
    
    labelled_new_mask = labelled_spheres
    
    
    otsu_contacts= ds.force_pdf(I2, labelled_new_mask,threshold_contact)

    thr_low = threshold_contact - 10
    thr_high = threshold_contact + 10
    delta_thr = 4
    low_threshold = range(thr_low,thr_high,delta_thr)
    histograms = [ds.force_pdf(I2,labelled_new_mask,intensity) for intensity in low_threshold]

    for j in range(4):
        scaling = ds.force_plot(histograms,otsu_contacts,scaling=j,threshold_range=low_threshold,
                                otsu_threshold=threshold_contact,fitting=False)
        plt.savefig(folder+"t%01d/c2/pdf_scaling_test_%s_maskThr%02d.pdf"%(t,scaling.real,mask_thr))
                    
    for k in range(len(histograms)):
        histogram = np.array(histograms)[k]
        fname = folder+'t%01d/c2/histograms/maskThr%02d_force_histograms_%s'%(t,mask_thr,histogram[-1])
        ds.write_histogram_file(fname,histogram)

    fname = folder+'t%01d/c2/histograms/maskThr%02d_force_histograms_otsu%s'%(t,mask_thr,np.round(otsu_contacts[-1],0))
    ds.write_histogram_file(fname,otsu_contacts)
                    
    coms, maxs,shifted_maxs,contact_size,residualContacts,residualLabels =ds.label_contactCentre(I2,
                                                            labelled_new_mask,threshold_contact)[:-1]
    ds.contact_centre_save(np.array(coms),contact_size,
                           folder+'t%01d/c2/particle_middle/c2enters_Contacts_coms_new_maskThr%02d.txt'%(t,mask_thr))
    ds.contact_centre_save(np.array(maxs),contact_size,
                           folder+'t%01d/c2/particle_middle/c2enters_Contacts_maxs_new_maskThr%02d.txt'%(t,mask_thr))
    ds.contact_centre_save(shifted_maxs,contact_size,
                           folder+'t%01d/c2/particle_middle/c2enters_Contacts_shiftedmaxs_new_maskThr%02d.txt'%(t,mask_thr))
    

    
    # Load the coordination of all contacts
    contacts= np.loadtxt(folder+'t%01d/c2/particle_middle/c2enters_Contacts_coms_new_maskThr00.txt'%t)
    removeLarge = contacts[:,-1]<400        #### !!!!!!
    contact_cen_new = contacts[removeLarge][:,:3]
    contact_cen = deepcopy(contact_cen_new)
    contact_size = contacts[removeLarge][:,-1]
    
#     mlab.points3d(p_cen[:,0],p_cen[:,1],p_cen[:,2],scale_factor=15,color=(0,0,0))#,p_cen[:,3])
#     xc = contact_cen[:,0]
#     yc = contact_cen[:,1]
#     zc = contact_cen[:,2]
#     mlab.points3d(xc,yc,zc, scale_factor=5,color=(0,0,1))
#     mlab.show()
    
#     # compute distances between all contacts and particles
    dists=cdist(p_cen[:,:3], contact_cen)
    dists.shape

    # which two particles have a common contact point
    p1_cen, p2_cen, p_tb_connected, connections, forces = analysis.link_particles(
                                dists,contact_cen,contact_size,p_cen)

    #if a contact is not collinear with two adjacent particles, put three of them collinear
    force_vector = analysis.find_force_vector(contact_cen,p1_cen, p2_cen,forces)
    # np.savetxt(folder+'t%01d/c2/particle_middle/force_vector.txt'%t ,force_vector)

    # calculate number of contacts per particle
    # sum of force per particle with magnitude or not
    ncontacts,local_sum,local_sum_moduli,order,local_f_list,local_f_sum = analysis.n_contacts_pparticle(
                                                    p_tb_connected,force_vector,p_cen)
    
    coordinations = analysis.coordination(p_cen,maxdistance)
    np.savetxt(folder+'t%01d/c2/particle_middle/nContacts.txt'%t,ncontacts)
    np.savetxt(folder+'t%01d/c2/particle_middle/nNeighbors.txt'%t,coordinations)
    
    #### plot p(N) ,P(Z)   
    f = plt.figure(figsize=(11,4))
    ax1 = f.add_subplot(121)
    ax1.hist(ncontacts[ncontacts<13],histtype='step',normed=True)
    mean_Z = ncontacts[ncontacts<13].mean()
    ax1.text(0.8,0.9,'<$Z$> = %s'%np.round(mean_Z,1), ha='center', va='center', transform=ax1.transAxes,size=12)
    ax1.set_xlabel('$Z$'),ax1.set_ylabel('$P(Z)$')
    ax1.set_title('Number of contacts, Z')

    ax2 = f.add_subplot(122)
    ax2.hist(coordinations,histtype='step',normed=True)
    mean_N = coordinations.mean()
    ax2.text(0.8,0.9,'<$N$> = %s'%np.round(mean_N,1), ha='center', va='center', transform=ax2.transAxes,size=12)
    ax2.set_xlabel('$N$'),ax2.set_ylabel('$P(N)$')
    ax2.set_title('Number of neighbors, N')

    plt.savefig(folder+'t%01d/c2/particle_middle/P(N)_P(Z).pdf'%t)
    
    #### plot local sum   
    f = plt.figure(figsize=(18,4))
    ax1 = f.add_subplot(131)
    ax1.hist(force_vector[:,-3],bins=30)
    ax1.set_xlabel('Force vectors'), ax1.set_ylabel('')
    ax1.legend(loc=2)

    ax2 = f.add_subplot(132)
    ax2.plot(local_sum[order],label='Vector sum of forces')
    ax2.plot(local_sum_moduli[order],label = 'Magnitude sum of forces')
    ax2.set_xlabel('Droplet Index'), ax2.set_ylabel('')
    ax2.legend(loc=2)

    # number of contacts distribution along z direction
    ax3 = f.add_subplot(133)
    bin_num=20
    plt.hist(contact_cen[:,2], bins=bin_num,color='g',alpha=0.3,label='Contacts')
    plt.xlabel('Z - direction')
    plt.ylabel('$Z$')
    plt.plot(imageC2.mean(axis=(1,2))*10, lw=2,color='gray', label='Intensity average')
    plt.xlim(0,imageC2.shape[0])
    plt.legend(loc=1)

    plt.savefig(folder+'t%01d/c2/particle_middle/forceSum_NcZ.pdf'%t)
    
    # compute compression and tension difference
    comp_tens_df = analysis.compression_tension(coordinations,ncontacts)
    fig.write_xyz_file_p(folder + 't%01d/c2/particle_middle/compression_tension'%t,
                                    p_cen,Property =comp_tens_df)
    # particles have high contact number, more than 3 contacts
    pcen_highcontacts = p_cen[ncontacts>=3]
    fig.write_xyz_file(folder + 't%01d/c2/particle_middle/pcen_highcontacts'%t,p_cen)
    fig.write_xyz_file_p(folder + 't%01d/c2/particle_middle/particle_num_of_contacts_label'%t,
                                    p_cen,Property =ncontacts)
    fig.write_xyz_file_p(folder + 't%01d/c2/particle_middle/particle_coordination_label'%t,
                                    p_cen,Property =coordinations)

    good_ncontacts = ncontacts<13
    hists = np.stack((ncontacts[good_ncontacts],coordinations[good_ncontacts]))
    # hists = np.stack((ncontacts[pcen_boxmiddle],
    #                   coordinations[pcen_boxmiddle]))
    nNeigh_ave = coordinations.sum().astype(float)/coordinations.shape[0]
    nCon_ave=ncontacts.sum().astype(float)/ncontacts.shape[0]
    labels = ['Z','N']
    text = [['N_p',p_cen.shape[0]],
           ['N_c',contact_cen.shape[0]],
           ['A_c',nCon_ave],
           ['A_{coor}',nNeigh_ave]]

    fig.histogram(hists,bins_num=np.array((12,12)),xlabel='$N$',ylabel='Counts',
                  labels=labels,
                  directory =folder+'t%01d/c2/particle_middle/'%t )
    plt.savefig(folder+'t%01d/c2/particle_middle/N_and_Z.pdf'%t)

    # Detect clusters
    import scipy.cluster.hierarchy as hac
    Cluster = hac.fcluster(hac.linkage(pcen_all[:,:3],'single'),maxdistance,'distance')
    print ('number of cluster detected is', Cluster.max())
    fig.write_xyz_file_p(folder + 't%01d/c2/particle_middle/Cluster_detection_allParticles'%t,
                                    p_cen,Property =Cluster)
    # check if the gel is percolating
    analysis.percolation(Cluster,imageC2,pcen_all)

    #plot cluster size distribution
    clusters, counts = np.unique(Cluster, return_counts=True)
    cluster_size, cluster_counts = np.unique(counts,return_counts=True)
    fig.plot_scatter(clusters,counts,#marker=r'$\clubsuit$',
                                  xlabel='Labelled Clusters',
                                  ylabel='Number of Particles',
                                  yscale='log')
    plt.savefig(folder+'t%01d/c2/particle_middle/cluster_detection_allParticles.pdf'%t)  
    
    

    # which two particles have a common contact point
    p1_cen, p2_cen, p_tb_connected, connections, forces = analysis.link_particles(
                                dists,contact_cen,contact_size,p_cen)
    
    coordinations = analysis.coordination(p_cen,maxdistance)
    np.savetxt(folder+'t%01d/c2/particle_middle/nNeighbors.txt'%t,coordinations)
    
    #if a contact is not collinear with two adjacent particles, put three of them collinear
    force_vector = analysis.find_force_vector(contact_cen,p1_cen, p2_cen,forces)
    np.savetxt(folder+'t%01d/c2/particle_middle/force_vector.txt'%t ,force_vector)

    # calculate number of contacts per particle
    # sum of force per particle with magnitude or not
    ncontacts,local_sum,local_sum_moduli,order,local_f_list,local_f_sum = analysis.n_contacts_pparticle(
                                                    p_tb_connected,force_vector,p_cen)
    np.savetxt(folder + 't%01d/c2/particle_middle/nContacts.txt'%t,ncontacts)

    ##### f direction is contacts to particles, r direction is opposite
    stress_tensor, parToCon_list = ca.force_chains3d.compute_stress(p_cen,contact_cen,local_f_list,dists)
    major_stress, minor_stress, eigvals,stress_trace,anisotropy = ca.force_chains3d.principal_stress(stress_tensor)

    np.savetxt(folder + 't%01d/c2/particle_middle/stress_tensor/minor_stress.txt'%t,np.array(minor_stress).real)
    np.savetxt(folder + 't%01d/c2/particle_middle/stress_tensor/major_stress.txt'%t,np.array(major_stress).real)
    np.savetxt(folder + 't%01d/c2/particle_middle/stress_tensor/stress_trace.txt'%t,np.array(stress_trace).real)
    np.savetxt(folder + 't%01d/c2/particle_middle/stress_tensor/anisotropy.txt'%t,anisotropy)
    np.savetxt(folder + 't%01d/c2/particle_middle/stress_tensor/particle_stress_index.txt'%t,p_inbox)
    
    s = [x[1] for x in stress_tensor]
    cPickle.dump(s, open(folder + 't%01d/c2/particle_middle/stress_tensor/stress_tensor.pkl'%t,'wb'))
#     np.savetxt(folder + 't%01d/c2/particle_middle/stress_tensor/stress_tensor.txt'%t,np.array(stress_tensor))
    
    plt.hist(np.real(minor_stress),bins=20,histtype='step')
    plt.yscale('log')
    plt.xlabel('Minor Stress'),plt.ylabel('pdf')
    plt.savefig(folder + 't%01d/c2/particle_middle/stress_tensor/minor_stress_pdf.pdf'%t)
    plt.figure()
    plt.hist(np.real(major_stress),bins=20,histtype='step')
    plt.yscale('log')
    plt.xlabel('Major Stress'),plt.ylabel('pdf')
    plt.savefig(folder + 't%01d/c2/particle_middle/stress_tensor/major_stress_pdf.pdf'%t)
    plt.figure()
    plt.hist(stress_trace,bins=20,histtype='step')
    plt.yscale('log')
    plt.xlabel('Stress Trace'),plt.ylabel('pdf')
    plt.savefig(folder + 't%01d/c2/particle_middle/stress_tensor/stress_trace_pdf.pdf'%t)
      
    np.savetxt(folder + 't0/c2/particle_middle/local_sum_manual.txt',local_sum)

    fig.write_xyz_file_p(folder + 't0/c2/particle_middle/particle_forceMagnitude_manual',
                                    p_cen[:,:3],Property =local_sum)

    fig.write_xyz_file_p(folder + 't0/c2/particle_middle/particle_minor_stress_manual',
                                    p_cen[:,:3],Property =minor_stress)

    largeForce = local_sum>=local_sum.mean()
    fig.write_xyz_file_p(folder + 't0/c2/particle_middle/particle_forceMagnitude_large_manual',
                                    p_cen[largeForce][:,:3],Property =local_sum[largeForce])

#     major_angle = np.array(major_theta)*360/np.pi
#     write_xyz_file(folder + 'c2/particle_middle/particle_major_angle',
#                                     p_cen[:,:2],Property =major_angle)
#     largeForce = local_sum>=local_sum.mean()
#     fig.write_xyz_file(folder + 'c2/particle_middle/particle_major_angle_large',
#                                     p_cen[largeForce][:,:2],Property =major_angle[largeForce])
    
    neighbours_distance, coordinations = ds.coordination(p_cen[:,:3],maxdistance=maxdistance)[:-1]
    plt.figure()
    plt.hist(ncontacts,bins=12,alpha=0.8,label='$Nz$')
    plt.hist(coordinations,bins=12,alpha=0.8,label='$CN$')
    plt.xlim(0,15)
    plt.legend()
    
    neighbours_distance=np.array(neighbours_distance)
    non_alone = np.where(coordinations>=1)[0]
    np.savetxt(folder + 't%01d/c2/particle_middle/non_alone.txt'%t,non_alone)
    
    highStress_neighbours=[] 
    for i in non_alone:  
        idx = np.where(neighbours_distance[:,0]==i)[0]
        newneigh = []
        for j in idx:
            neighbours = neighbours_distance[j][1]
            newneigh.append(neighbours)
        highStress_neighbours.append((i,newneigh))

    chain_computed = ca.force_chains3d.find_chains(p_cen,
                            coordinations,highStress_neighbours,minor_stress,eigvals,threshold=45)
    # chain_color, uu, vv,ww,XYZ = ca.force_chains3d.find_chains(p_cen,ncontacts,
    #                 highStress_neighbours,minor_stress,eigvals)
    chain_color = chain_computed[0]
    uu,vv,ww = chain_computed[1],chain_computed[2],chain_computed[3]
    XYZ,pairs,Neigh = chain_computed[-3],chain_computed[-2],chain_computed[-1]

    chainlength,labelled_chain,coloredP,length = ca.force_chains3d.color_chains(
                                            chain_color,p_cen,non_alone)
    NP_inChain = length[length>2].shape[0]
    
    to_delete = []
    for i in range(len(pairs)):
        for j in range(i+1,len(pairs)):
            if pairs[i] == pairs[j][::-1]:
                to_delete.append(j)

    new_pairs = np.delete(pairs,to_delete,axis=0)
    long_pairs =[]
    for i in range(len(new_pairs)):
        Lchain = length[new_pairs[i][0]]
        if Lchain >3:
            long_pairs.append((new_pairs[i][0],new_pairs[i][1],Lchain))

    pair_index = []
    for i in range(len(long_pairs)):
        p1 = new_pairs[i][0]
        p2 = new_pairs[i][1]
        old_p1 = non_alone[p1]
        old_p2 = non_alone[p2]
        c_idx =np.where(p_tb_connected[:,1]==old_p1)[0]
        if p_tb_connected[c_idx].shape[0]>0:
            connect_p = p_tb_connected[c_idx]
            matched = [p==old_p2 for p in connect_p[:,-1]]
            if any(matched):
                size = connect_p[matched][0][0]
                pair_index.append((p1,p2,size))
    pair_index = np.array(pair_index)

    pair_index_oldp = []
    for i in range(len(long_pairs)):
        p1 = long_pairs[i][0]
        p2 = long_pairs[i][1]
        old_p1 = non_alone[p1]
        old_p2 = non_alone[p2]

        pair_index_oldp.append((old_p1,old_p2))
    pair_index_oldp = np.array(pair_index_oldp)

    np.savetxt(folder+'t%01d/c2/particle_middle/test/p_non_alone.txt'%t,p_cen[non_alone])
    np.savetxt(folder+'t%01d/c2/particle_middle/test/labelled_chain.txt'%t,labelled_chain)
    np.savetxt(folder+'t%01d/c2/particle_middle/test/new_pairs.txt'%t,new_pairs)
    np.savetxt(folder+'t%01d/c2/particle_middle/test/pair_index.txt'%t,pair_index)

    np.savetxt(folder+'TCC/chainlength/chainlength_t%02d.txt'%t,length)
    plt.hist(length,bins=10,normed=True)
    plt.yscale('log')
    plt.xlabel('chain size [particles]'),plt.ylabel('pdf')
    plt.savefig(folder+'t%01d/chainlength_distribution.pdf'%t)